In [1]:
from functools import singledispatch

## Currently proposed implementation (simplified)

Here there is just one `Explanation` object which is the same for any explanation method. To distinguish between different explanations one can use the `name` attribute. Here, we use the `name` attribute to dynamically dispatch a call to the generic `show` function to the conrete `show_concrete` function.

Features:
 - top level e.g. `alibi.show` function for visualizing any explanation
 - the explanation method `exp.show` just calls `alibi.show(self)`
 - `alibi.show(exp)` dispatches to `show_concrete` dynamically based on `exp.name`

Issues:
 - How do we make the docstring in `show_concrete` show up when calling `help(concrete_explanation.show)` ? This is likely impossible as we dispatch on a runtime attribute. **This is a crucial requirement as concrete methods will take different arguments depending on the explanation.**
 - The top level `show` function can't by design show explanation specific arguments in any case
 
Advantages:
 - No need to subclass `Explanation`
 
Disadvantages:
 - Methods shared between many different explanations, e.g. some might not make sense but would still have to be defined at the `Explanation` implementation even if only to raise `NotImplementedError`
 - How to dispatch methods to concrete implementations but retain the docstrings when calling `help`?

In [2]:
class Explanation:
    
    def __init__(self, name):
        self.name = name
        
    def show(self, **kwargs):
        """
        Explanation bound method `show` docstring
        """
        return show(self, **kwargs)


def show_concrete(exp: Explanation, **kwargs):
    """
    Concrete `show` docstring
    """
    
    print('show_concrete called')


DISPATCH_DICT = {'concrete': show_concrete}


def show(exp: Explanation, **kwargs):
    """
    Top level `show` docstring
    """
    try:
        DISPATCH_DICT[exp.name](exp, **kwargs)
    except KeyError:
        raise NotImplementedError(f"Visualization for `{exp.name}` not implemented")

In [3]:
concrete = Explanation(name='concrete')

In [4]:
concrete.show()

show_concrete called


In [5]:
# actual implementation docstring not shown, NEED to fix
help(concrete.show)

Help on method show in module __main__:

show(**kwargs) method of __main__.Explanation instance
    Explanation bound method `show` docstring



In [6]:
# actual implementation docstring not shown, CANNOT FIX
help(show)

Help on function show in module __main__:

show(exp: __main__.Explanation, **kwargs)
    Top level `show` docstring



## Explicit subclassing

Here we have a traditional approach where the base `Explanation` class only defines an interface and is not meant to be instantiated. What is instantiated eventually is a `ConcreteExplanation` object which overrides the `show` method and the docstring shows correctly.

The top level `show` function can now be implemented using `@singledispatch` decorator instead of manually dispatching on the `name` attribute. However, it still can't by design show explanation specific arguments in any case. This raises the question whether a top level `show` function is of any use.

Advantages:
 - Explicit overriding of `show` method for explanation subclasses
 - No `mypy` issues with "non-existent attributes" when trying to write a `show_concrete` implementation on a generic `Explanation` object
 - In the future allows for more specialized/granular methods for explanations, e.g. `show_all` might make sense for only a subset of explanations

Disadvantages:
 - For every explanation type need to write a subclass

In [7]:
class Explanation:
    def __init__(self, name):
        self.name = name
        
    def show(self):
        """
        Explanation bound method `show` docstring
        """
        raise NotImplementedError(f"Visualization for {self.name} is not available")

class ConcreteExplanation(Explanation):
    
    def __init__(self, name):
        super().__init__(name)
        
    def show(self):
        """
        Concrete `show` docstring
        """
        print("show_concrete called")
        
@singledispatch
def show(explanation):
    """
    Top level `show` docstring
    """
    raise NotImplementedError()
    
@show.register
def _(explanation: ConcreteExplanation):
    """
    Dispatched `show` docstring
    """
    print("dispatched `show` called")

In [8]:
concrete2 = ConcreteExplanation(name='concrete2')

In [9]:
concrete2.show()

show_concrete called


In [10]:
# actual implementation docstring is shown, FIXED
help(concrete2.show)

Help on method show in module __main__:

show() method of __main__.ConcreteExplanation instance
    Concrete `show` docstring



In [11]:
# actual implementation docstring not shown, CANNOT FIX
help(show)

Help on function show in module __main__:

show(explanation)
    Top level `show` docstring



## Proposal

In light of the experiments above I would propose:
 - moving to subclassing for concrete explanations as this would enable more flexibility in the subsequent bound method design
 - not implementing a top level `alibi.show` function as the use cases seem limited especially with no way to display the correct docstring